In [1]:
import torch
import numpy as np

In [2]:
import torch.nn as nn

In [3]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

In [4]:
input=torch.from_numpy(inputs)
target=torch.from_numpy(targets)

In [5]:
input

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

In [6]:
from torch.utils.data import TensorDataset

In [7]:
train_ds=TensorDataset(input,target)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [8]:
from torch.utils.data import DataLoader

In [9]:
bs=5
train_dl=DataLoader(train_ds,bs,shuffle=True)

In [10]:
for xb,yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[91., 87., 65.],
        [73., 67., 43.],
        [68., 96., 71.],
        [69., 96., 70.],
        [92., 87., 64.]])
tensor([[ 80., 102.],
        [ 56.,  70.],
        [104., 118.],
        [103., 119.],
        [ 82., 100.]])


In [11]:
model=nn.Linear(3,2) # 3 inputs and 2 outputs
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.1004,  0.4508, -0.1331],
        [-0.1155,  0.3330,  0.1455]], requires_grad=True)
Parameter containing:
tensor([-0.4219, -0.1062], requires_grad=True)


In [12]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.1004,  0.4508, -0.1331],
         [-0.1155,  0.3330,  0.1455]], requires_grad=True),
 Parameter containing:
 tensor([-0.4219, -0.1062], requires_grad=True)]

In [13]:
preds=model(input)
preds

tensor([[16.7255, 20.0344],
        [21.5893, 28.0055],
        [43.5245, 42.9129],
        [ 3.7929,  7.8200],
        [26.6067, 34.0833],
        [16.1743, 19.5859],
        [21.0055, 27.8180],
        [43.2910, 42.9430],
        [ 4.3441,  8.2685],
        [26.5741, 34.3443],
        [16.1417, 19.8469],
        [21.0381, 27.5570],
        [44.1083, 43.1004],
        [ 3.8255,  7.5590],
        [27.1579, 34.5318]], grad_fn=<AddmmBackward>)

In [14]:
import torch.nn.functional as F

In [15]:
loss_fn=F.mse_loss

In [16]:
loss=loss_fn(preds,target)
loss

tensor(4085.4861, grad_fn=<MseLossBackward>)

In [17]:
opt=torch.optim.SGD(model.parameters(),lr=1e-5)

In [18]:
def fit(num_epochs,model,loss_fn,opt,train_dl):
    for epoch in range(num_epochs):
        for xb,yb in train_dl:
            preds=model(xb)
            loss_fn=F.mse_loss
            loss=loss_fn(preds,yb)
            loss.backward()
            opt.step() # update the weights and biasis
            opt.zero_grad() # reset the gratdiants to zero
        if(epoch+1)%10 ==0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')




In [19]:
fit(100,model,loss_fn,opt,train_dl)

Epoch [10/100], Loss: 158.63681030273438
Epoch [20/100], Loss: 122.2944107055664
Epoch [30/100], Loss: 85.65492248535156
Epoch [40/100], Loss: 105.11962890625
Epoch [50/100], Loss: 61.9456901550293
Epoch [60/100], Loss: 45.76727294921875
Epoch [70/100], Loss: 38.04197311401367
Epoch [80/100], Loss: 59.22895431518555
Epoch [90/100], Loss: 15.398846626281738
Epoch [100/100], Loss: 13.661625862121582


In [24]:
preds=model(input)
preds

tensor([[ 57.8884,  71.2249],
        [ 79.0922,  98.1772],
        [123.3547, 136.7235],
        [ 26.3302,  42.4693],
        [ 93.4470, 111.6288],
        [ 56.6816,  70.1862],
        [ 78.3568,  97.8007],
        [123.3581, 137.0935],
        [ 27.5370,  43.5080],
        [ 93.9184, 112.2910],
        [ 57.1530,  70.8484],
        [ 77.8854,  97.1385],
        [124.0901, 137.1000],
        [ 25.8588,  41.8071],
        [ 94.6538, 112.6675]], grad_fn=<AddmmBackward>)

In [23]:
targets

array([[ 56.,  70.],
       [ 81., 101.],
       [119., 133.],
       [ 22.,  37.],
       [103., 119.],
       [ 57.,  69.],
       [ 80., 102.],
       [118., 132.],
       [ 21.,  38.],
       [104., 118.],
       [ 57.,  69.],
       [ 82., 100.],
       [118., 134.],
       [ 20.,  38.],
       [102., 120.]], dtype=float32)